In this notebook we'll be collecting data from the EPA AQS api (https://aqs.epa.gov/aqsweb/documents/data_api.html) for all of the available monitoring sites in Cook county. According to the wikipedia page for AQI values in the United States (https://en.wikipedia.org/wiki/Air_quality_index#United_States) can be based on a number of different measurements. Namely, the $O_3$ (8-hr and 1-hr), $PM_{2.5}$ (24-hr), $PM_{10}$ (24-hr), $CO$ (8-hr), $SO_2$ (1-hr and 24-hr), and $NO_2$ (1-hr) concentrations. The Wikipedia article has details for which values to use in certain scenarios for calculating AQI.

In [6]:
import numpy as np
import pandas as pd
import requests
import json

In [20]:
# api access variables
# state and county codes obtained from the list sample queries on the
url_head = 'https://aqs.epa.gov/data/api'
email = 'garrett.l.ducharme@gmail.com'
with open('C:/Users/ducha/.api_keys/aqs.txt') as f:
    key = json.load(f)["key"]
state = '17'
county = '031'

In [35]:
# Obtaining monitoring sites for cook county
end_point = 'list/sitesByCounty'
url = f'{url_head}/{end_point}?email={email}&key={key}&state={state}&county={county}'
site_codes = requests.get(url)
sites = pd.DataFrame(json.loads(site_codes.text)['Data'])
sites.head()

,code,value_represented
0,0001,VILLAGE GARAGE
1,0002,None
2,0003,None
3,0004,None
4,0005,None


In [42]:
# Obtaining the valid parameter list
end_point = 'list/classes'
url = f'{url_head}/{end_point}?email={email}&key={key}'
param_codes = requests.get(url)
params = pd.DataFrame(json.loads(param_codes.text)['Data'])
params

,code,value_represented
0,AIRNOW MAPS,The parameters represented on AirNow maps (881...
1,ALL,Select all Parameters Available
2,AQI POLLUTANTS,Pollutants that have an AQI Defined
3,CORE_HAPS,Urban Air Toxic Pollutants
4,CRITERIA,Criteria Pollutants
5,CSN DART,List of CSN speciation parameters to populate ...
6,FORECAST,Parameters routinely extracted by AirNow (STI)
7,HAPS,Hazardous Air Pollutants
8,IMPROVE CARBON,IMPROVE Carbon Parameters
9,IMPROVE_SPECIATION,PM2.5 Speciated Parameters Measured at IMPROVE...


All of the polutants that we need to calculate the AQI are contained within the 'CRITERIA' class.

In [41]:
# Obtain all parameters in the criteria class
end_point = 'list/parametersByClass'
pc = 'CRITERIA'
url = f'{url_head}/{end_point}?email={email}&key={key}&pc={pc}'
criteria_codes = requests.get(url)
criteria = pd.DataFrame(json.loads(criteria_codes.text)['Data'])
criteria.head()

,code,value_represented
0,14129,Lead (TSP) LC
1,42101,Carbon monoxide
2,42401,Sulfur dioxide
3,42602,Nitrogen dioxide (NO2)
4,44201,Ozone


All meteorological data is contained in the 'MET' class.

In [43]:
end_point = 'list/parametersByClass'
pc = 'MET'
url = f'{url_head}/{end_point}?email={email}&key={key}&pc={pc}'
met_codes = requests.get(url)
met = pd.DataFrame(json.loads(cri_codes.text)['Data'])
met.head()

,code,value_represented
0,14129,Lead (TSP) LC
1,42101,Carbon monoxide
2,42401,Sulfur dioxide
3,42602,Nitrogen dioxide (NO2)
4,44201,Ozone
